
## **Öğretmen Maaşı Tahmini Projesi**:
Veri Analizi ve Makine Öğrenmesi Modelleri

Bu çalışmada öğretmen maaş verilerini analiz ederek gelecekteki maaş beklentilerini tahmin etmek için makine öğrenmesi modelleri kıyaslanarak en iyi model kullanıldı. Veriler 'İstanbul Öğretmenleri' adlı https://t.me/+rI_ypeTTrC1hMTI0
 telegram sayfasında anketlerle oluşturuşup resim, pdf dosyası biçiminde paylaşımıştır. Paylaşımlar excele dönüştürülerek kullanılabilir verisetine dönüştürülmüştür. Çalışmam aşağıdaki adımları içermektedir:

1.  **Veri Yükleme ve Hazırlık**: Maaş verilerini içeren Excel dosyası (`maas.xlsx`) yüklenmiş ve analiz için gerekli sütunlar seçilerek eksik değerler temizlenmiştir.
2.  **Makine Öğrenmesi Modellerinin Eğitimi**: Farklı regresyon modelleri (Karar Ağacı, Random Forest, KNN Regressor, Doğrusal Regresyon) belirlenen özellikler (`mevcut maaş ortalaması`, `12 Aylık Ortalamalara Göre Haziran Ayı Tüfe Oranı %`, `MEB Temmuz Ayı 1/4 Kademe Öğretmen Haftanın Bir Günü Maaşı`) kullanılarak gelecek yılın toplam ortalama maaş beklentisini tahmin etmek üzere eğitilmiş ve performansları değerlendirilmiştir.
3.  **Gelecek Yıl Maaşlarının Tahmini**: Eğitilen modellerden biri (Doğrusal Regresyon), 2022'den 2026'ya kadar olan yıllar için branş bazında tahmini maaşları hesaplamak üzere kullanılmıştır. Bu tahminler özet bir tabloda sunulmuştur.
4.  **Branş Bazında Maaş Değişiminin Görselleştirilmesi**: Belirli branşlar (örneğin, Matematik Lise) için yıllara göre mevcut maaş, bir sonraki yılın beklenen maaşı ve modelin tahmin ettiği maaş arasındaki değişimi gösteren çizgi grafikleri oluşturulmuştur.

Bu çalışma, mevcut verilere dayanarak öğretmen maaşlarındaki eğilimleri anlamak ve geleceğe yönelik olası senaryolar hakkında fikir edinmek için bir başlangıç noktası sunmaktadır.

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Excel dosyasını yükle
df = pd.read_excel(r"maas.xlsx")

# Özellikler (X) ve hedef değişken (y) tanımlayalım
# Hedef değişkenimiz 'gelecek yıl toplam ortalama maaş beklentisi' olacak.
# Özelliklerimiz ise 'mevcut maaş ortalaması', '12 Aylık Ortalamalara Göre Haziran Ayı Tüfe Oranı %', 'MEB Temmuz Ayı 1/4 Kademe Öğretmen Haftanın Bir Günü Maaşı'
X = df[['mevcut maaş ortalaması',
        '12 Aylık Ortalamalara Göre Haziran Ayı Tüfe Oranı %',
        'MEB Temmuz Ayı 1/4 Kademe Öğretmen Haftanın Bir Günü Maaşı']].copy()
y = df['gelecek yıl toplam ortalama maaş beklentisi'].copy()

# Eksik değerleri temizleyelim (modeller için gerekli)
# Sadece X ve y'deki eksik değerleri siliyoruz
data = pd.concat([X, y], axis=1).dropna()
X = data[['mevcut maaş ortalaması',
          '12 Aylık Ortalamalara Göre Haziran Ayı Tüfe Oranı %',
          'MEB Temmuz Ayı 1/4 Kademe Öğretmen Haftanın Bir Günü Maaşı']]
y = data['gelecek yıl toplam ortalama maaş beklentisi']


# Sayısal sütunları tanımlayalım
numerical_cols = X.columns.tolist() # All features are numerical in this case

# Pipeline: Ön işleme (ölçeklendirme) + Model
# Kategorik sütun olmadığı için OneHotEncoder'a gerek yok
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols)
], remainder='passthrough') # Keep other columns (none in this case)


# Modelleri tanımlayalım
models = {
    'Karar Ağacı': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'KNN Regressor': KNeighborsRegressor(n_neighbors=5),
    'Linear Regression': LinearRegression() # Add Linear Regression as well
}

# Eğitim ve test verisine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sonuçları tutmak için tablo
results = []

# Her bir model için eğitim ve test
for name, model in models.items():
    pipe = Pipeline(steps=[('preprocess', preprocessor), ('model', model)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        'Model': name,
        'MSE': round(mse, 2),
        'MAE': round(mae, 2),
        'R² Skoru': round(r2, 3)
    })

# DataFrame'e dökelim
results_df = pd.DataFrame(results)
print(results_df)

               Model         MSE     MAE  R² Skoru
0        Karar Ağacı   582143.11  467.22     0.983
1      Random Forest   415073.25  360.32     0.988
2      KNN Regressor   413839.29  436.18     0.988
3  Linear Regression  1066300.23  855.56     0.969




*   **MSE** ve **MAE** değerlerinin daha düşük olması modelin daha iyi performans gösterdiği anlamına gelir. Bu metriklerde en düşük değerlere sahip modeller KNN Regressor ve Random Forest  olarak görünüyor.

*   **R² Skoru**'nun 1'e daha yakın olması modelin veriye daha iyi uyduğunu gösterir. Random Forest ve KNN Regressor en yüksek R² skorlarına (0.988) sahip.

Genel olarak, Random Forest ve KNN Regressor modelleri en iyi performansı göstermiştir. Bende bu sonuçlara göre **Random Forest** modelini tercih ediyorum.

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

def tahmin_tablo(df, prediction_years):

    df_tahminler = df.copy()
    tum_tahminler_data = pd.DataFrame() # Pivoting için daha sonra tahmini verileri toplamak üzere

    for pred_year in prediction_years:
        # Tahmin yılından önceki yıla kadar olan verilerle modeli eğit
        train_data = df_tahminler[df_tahminler['Sene'] <= pred_year - 1].copy()

        if not train_data.empty :
            X_train = train_data[['mevcut maaş ortalaması',
                                  '12 Aylık Ortalamalara Göre Haziran Ayı Tüfe Oranı %',
                                  'MEB Temmuz Ayı 1/4 Kademe Öğretmen Haftanın Bir Günü Maaşı']]
            y_train = train_data['gelecek yıl toplam ortalama maaş beklentisi']

            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)

            # Tahmin yılı için, bir önceki yıla ait verileri kullanarak tahmin yap
            predict_data = df_tahminler[df_tahminler['Sene'] == pred_year - 1].copy()

            if not predict_data.empty:
                X_predict = predict_data[['mevcut maaş ortalaması',
                                          '12 Aylık Ortalamalara Göre Haziran Ayı Tüfe Oranı %',
                                          'MEB Temmuz Ayı 1/4 Kademe Öğretmen Haftanın Bir Günü Maaşı']]
                predicted_salary = model.predict(X_predict).round(0)

                # Pivoting için tahmin edilen maaşları yıl ve branş ile birlikte topla
                temp_tahmin_df = predict_data[['Branş']].copy()
                temp_tahmin_df['Sene'] = pred_year
                temp_tahmin_df['Tahmini Maaş'] = predicted_salary
                tum_tahminler_data = pd.concat([tum_tahminler_data, temp_tahmin_df], ignore_index=True)

    # Şimdi, toplanan tahmini verilerden özet tabloyu oluştur
    if not tum_tahminler_data.empty:
        pivot_table = tum_tahminler_data.pivot_table(index='Branş', columns='Sene', values='Tahmini Maaş', aggfunc='first')
        pivot_table.columns.name = None # 'Yıl' sütun adını kaldır
        return pivot_table
    else:
        return pd.DataFrame() # Tahmin verisi yoksa boş DataFrame döndür


# Fonksiyonu 2022'den 2026'ya kadar tahmini maaşları eklemek ve özet tabloyu almak için kullan
years_to_predict = range(2022, 2027)
tahmin_maas_pivot = tahmin_tablo(df, years_to_predict)

# Özet tabloyu görüntüle
print("Yıllara Göre Öğretmen Maaşları Tahmini")
display(tahmin_maas_pivot)

Yıllara Göre Öğretmen Maaşları Tahmini


,2022,2023,2024,2025,2026
Branş,,,,,
Almanca,NaN,NaN,NaN,8630.0,13860.0
Anaokulu Öğretmenliği,NaN,NaN,NaN,6421.0,10858.0
Beden Eğitimi,NaN,NaN,3285.0,6548.0,12038.0
Bilişim,NaN,NaN,NaN,7673.0,14166.0
Biyoloji,949.0,1591.0,5070.0,10209.0,18102.0
Coğrafya,836.0,1350.0,4012.0,8183.0,14120.0
Din Kültürü,NaN,1004.0,3072.0,6969.0,12579.0
Felsefe Grubu,782.0,1090.0,2953.0,7689.0,11191.0
Fen Bilgisi,730.0,1331.0,3797.0,7743.0,13685.0


In [18]:
# Eksik verisi olmayan branşların tespiti
if isinstance(tahmin_maas_pivot, pd.DataFrame):
    nan_olmayan_brans = tahmin_maas_pivot.dropna().index.tolist()
    print("Eksik tahmini maaş verisi olmayan branşlar:")
    print(nan_olmayan_brans)
else:
    print("tahmin_maas_pivot bir pandas DataFrame değil.")

Eksik tahmini maaş verisi olmayan branşlar:
['Biyoloji', 'Coğrafya', 'Felsefe Grubu', 'Fen Bilgisi', 'Fizik', 'Geometri', 'Kimya', 'Matematik Lise', 'Matematik Ortaokul', 'Rehberlik', 'Sosyal Bilgiler', 'Tarih', 'Türkçe Edebiyat', 'Türkçe Ortaokul', 'İngilizce']


## Her branş i̇çin grafik oluşturma

In [19]:
import altair as alt
import pandas as pd
import os # Dosya işlemleri için os modülünü import et

# Grafikleri kaydetmek için bir dizin oluştur
output_dir = "charts"
os.makedirs(output_dir, exist_ok=True)

# Tanımlanan branşlar üzerinde döngü yap ve her bir grafik için oluştur
for brans in nan_olmayan_brans:
    # Mevcut ve beklenen maaşları almak için orijinal dataframe'i geçerli branşa göre filtrele
    brans_beklenen = df[df['Branş'] == brans].copy()

    # Mevcut ve beklenen için ilgili sütunları seç
    brans_beklenen = brans_beklenen[['Sene', 'mevcut maaş ortalaması', 'gelecek yıl toplam ortalama maaş beklentisi']].copy()

    # Düzenlemeden önce sütun adlarını netlik için yeniden adlandır
    brans_beklenen.rename(columns={
        'mevcut maaş ortalaması': 'Mevcut Maaş',
        'gelecek yıl toplam ortalama maaş beklentisi': 'Beklenen Maaş (Sonraki Yıl)'
    }, inplace=True)

    # Mevcut ve beklenen maaş DataFrame'inin düzenlenmesi
    brans_melted_beklenen = brans_beklenen.melt(
        id_vars='Sene',
        value_vars=['Mevcut Maaş', 'Beklenen Maaş (Sonraki Yıl)'],
        var_name='Maaş Tipi',
        value_name='Maaş'
    ).dropna()

    # 'Beklenen Maaş (Sonraki Yıl)' Sene değerini temsil ettiği yıla ayarla
    brans_melted_beklenen.loc[brans_melted_beklenen['Maaş Tipi'] == 'Beklenen Maaş (Sonraki Yıl)', 'Sene'] = brans_melted_beklenen.loc[brans_melted_beklenen['Maaş Tipi'] == 'Beklenen Maaş (Sonraki Yıl)', 'Sene'] + 1


    # Pivot tablodan geçerli branş için tahmini maaşı al
    if brans in tahmin_maas_pivot.index:
        brans_tahmin_pivot = tahmin_maas_pivot.loc[[brans]].reset_index()

        # Tahmini maaş DataFrame'inin düzenlenmesi
        brans_melted_tahmin = brans_tahmin_pivot.melt(
            id_vars='Branş',
            value_vars=[col for col in brans_tahmin_pivot.columns if col != 'Branş'],
            var_name='Yıl',
            value_name='Maaş'
        ).dropna()

        # 'Yıl' sütunu zaten tam sayı yıl değerlerini içeriyor.
        brans_melted_tahmin['Sene'] = brans_melted_tahmin['Yıl'].astype(int)
        brans_melted_tahmin['Maaş Tipi'] = 'Tahmini Maaş'

        # Düzenlenen mevcut/beklenen verilerle eşleşmesi için sütunları seç ve yeniden sırala
        brans_melted_tahmin = brans_melted_tahmin[['Sene', 'Maaş Tipi', 'Maaş']]

    else:
        # Bu durum ideal olarak nan_olmayan_brans listesine göre olmamalı, ancak güvenlik için
        brans_melted_tahmin = pd.DataFrame(columns=['Sene', 'Maaş Tipi', 'Maaş'])


    # Tüm düzenlenen dataframeleri birleştir
    combined_plot_data = pd.concat([brans_melted_beklenen, brans_melted_tahmin], ignore_index=True)

    # Doğru sıralama ve çizim için 'Sene'nin tam sayı türü olduğundan emin ol
    combined_plot_data['Sene'] = combined_plot_data['Sene'].astype(int)

    # Doğru çizgi çizimi için yıla göre sırala
    combined_plot_data.sort_values(by='Sene', inplace=True)

    # Çizgi grafiği oluştur
    chart = alt.Chart(combined_plot_data).mark_line(point=True).encode(
        x=alt.X('Sene:O', title='Yıl'), # Yılları ayrı kategoriler olarak işlemek için ':O' kullan
        y=alt.Y('Maaş', title='Maaş'),
        color='Maaş Tipi',
        tooltip=['Sene', 'Maaş Tipi', alt.Tooltip('Maaş', format='.2f')] # Daha iyi okunabilirlik için araç ipucunu biçimlendir
    ).properties(
        title=f'{brans} Yıllara Göre Maaş Değişimi (Mevcut-Beklenen-Tahmin)',
        width=600 # Gerektiğinde genişliği ayarla
    )

    # Grafiği göster (Colab'da görüntülemek için)
    display(chart)

    # Grafiği JSON dosyası olarak kaydet (GitHub'da görüntülemek için)
    # Dosya adı için branş adındaki boşlukları ve özel karakterleri temizle
    file_brans = brans.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "")
    chart_file_path = os.path.join(output_dir, f"{file_brans}_maas_grafigi.json")
    chart.save(chart_file_path)

print(f"Grafikler '{output_dir}' dizinine JSON dosyaları olarak kaydedildi.")

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

Grafikler 'charts' dizinine JSON dosyaları olarak kaydedildi.


In [8]:
import altair as alt
import pandas as pd

# Tanımlanan branşlar üzerinde döngü yap ve her bir grafik için oluştur
for brans in nan_olmayan_brans:
    # Mevcut ve beklenen maaşları almak için orijinal dataframe'i geçerli branşa göre filtrele
    brans_beklenen = df[df['Branş'] == brans].copy()

    # Mevcut ve beklenen için ilgili sütunları seç
    brans_beklenen = brans_beklenen[['Sene', 'mevcut maaş ortalaması', 'gelecek yıl toplam ortalama maaş beklentisi']].copy()

    # Düzenlemeden önce sütun adlarını netlik için yeniden adlandır
    brans_beklenen.rename(columns={
        'mevcut maaş ortalaması': 'Mevcut Maaş',
        'gelecek yıl toplam ortalama maaş beklentisi': 'Beklenen Maaş (Sonraki Yıl)'
    }, inplace=True)

    # Mevcut ve beklenen maaş DataFrame'inin düzenlenmesi
    brans_melted_beklenen = brans_beklenen.melt(
        id_vars='Sene',
        value_vars=['Mevcut Maaş', 'Beklenen Maaş (Sonraki Yıl)'],
        var_name='Maaş Tipi',
        value_name='Maaş'
    ).dropna()

    # 'Beklenen Maaş (Sonraki Yıl)' Sene değerini temsil ettiği yıla ayarla
    brans_melted_beklenen.loc[brans_melted_beklenen['Maaş Tipi'] == 'Beklenen Maaş (Sonraki Yıl)', 'Sene'] = brans_melted_beklenen.loc[brans_melted_beklenen['Maaş Tipi'] == 'Beklenen Maaş (Sonraki Yıl)', 'Sene'] + 1


    # Pivot tablodan geçerli branş için tahmini maaşı al
    if brans in tahmin_maas_pivot.index:
        brans_tahmin_pivot = tahmin_maas_pivot.loc[[brans]].reset_index()

        # Tahmini maaş DataFrame'inin düzenlenmesi
        brans_melted_tahmin = brans_tahmin_pivot.melt(
            id_vars='Branş',
            value_vars=[col for col in brans_tahmin_pivot.columns if col != 'Branş'],
            var_name='Yıl',
            value_name='Maaş'
        ).dropna()

        # 'Yıl' sütunu zaten tam sayı yıl değerlerini içeriyor.
        brans_melted_tahmin['Sene'] = brans_melted_tahmin['Yıl'].astype(int)
        brans_melted_tahmin['Maaş Tipi'] = 'Tahmini Maaş'

        # Düzenlenen mevcut/beklenen verilerle eşleşmesi için sütunları seç ve yeniden sırala
        brans_melted_tahmin = brans_melted_tahmin[['Sene', 'Maaş Tipi', 'Maaş']]

    else:
        # Bu durum ideal olarak nan_olmayan_brans listesine göre olmamalı, ancak güvenlik için
        brans_melted_tahmin = pd.DataFrame(columns=['Sene', 'Maaş Tipi', 'Maaş'])


    # Tüm düzenlenen dataframeleri birleştir
    combined_plot_data = pd.concat([brans_melted_beklenen, brans_melted_tahmin], ignore_index=True)

    # Doğru sıralama ve çizim için 'Sene'nin tam sayı türü olduğundan emin ol
    combined_plot_data['Sene'] = combined_plot_data['Sene'].astype(int)

    # Doğru çizgi çizimi için yıla göre sırala
    combined_plot_data.sort_values(by='Sene', inplace=True)

    # Çizgi grafiği oluştur
    chart = alt.Chart(combined_plot_data).mark_line(point=True).encode(
        x=alt.X('Sene:O', title='Yıl'), # Yılları ayrı kategoriler olarak işlemek için ':O' kullan
        y=alt.Y('Maaş', title='Maaş'),
        color='Maaş Tipi',
        tooltip=['Sene', 'Maaş Tipi', alt.Tooltip('Maaş', format='.2f')] # Daha iyi okunabilirlik için araç ipucunu biçimlendir
    ).properties(
        title=f'{brans} Yıllara Göre Maaş Değişimi (Mevcut-Beklenen-Tahmin)',
        width=600 # Gerektiğinde genişliği ayarla
    )

    # Grafiği göster
    display(chart)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [15]:
import matplotlib.pyplot as plt
import os # Dosya işlemleri için os modülünü import et

# Grafikleri kaydetmek için bir dizin oluştur
output_dir = "charts"
os.makedirs(output_dir, exist_ok=True)

# Grafik oluşturma
plt.figure(figsize=(10, 6))
for maas_tipi in combined_plot_data['Maaş Tipi'].unique():
    subset = combined_plot_data[combined_plot_data['Maaş Tipi'] == maas_tipi]
    plt.plot(subset['Sene'], subset['Maaş'], marker='o', label=maas_tipi)

plt.title(f'{brans} Yıllara Göre Maaş Değişimi (Mevcut-Beklenen-Tahmin)')
plt.xlabel('Yıl')
plt.ylabel('Maaş (TL)')
plt.legend()
plt.grid(True)

# Grafiği PNG olarak kaydetme
filename = f"{output_dir}/{brans.replace(' ', '_')}_maas_degisimi.png"
plt.savefig(filename, bbox_inches='tight')
plt.close()

# Markdown için referans ekleme
print(f"![{brans} Maaş Değişimi]({filename})")

![İngilizce Maaş Değişimi](charts/İngilizce_maas_degisimi.png)


**ÖZET - SONUÇ**


Bu çalışmada İstanbulda özel kurumlarda öğretmenlik yapan öğretmenlerin telegram gruplarında 2021 - 2025 yılları arası kendi aralarında yaptıkları anketler sonucu birleştirilerek bir veriseti oluşturuldu. Verilerimiz 10 yıl öncesi tecrübeli ve 10 yıl üstü tecrübeli öğretmenlerin sayıları ve maaşları ortalamaları alınarak oluşturulduğundan bu veriler ortalama 10 yıllık tecrübeye sahip öğretmenlerin maaş tahmini yaptığını düşünmeliyiz.

Verisetimizde düzenlemeler yapılarak veriler, makine öğrenmesi modellerini karşılaştırarak en iyi performansı gösteren modeller belirlendi. Özellikle Random Forest ve KNN Regressor modellerinin yüksek R² skoru, modelin maaşlardaki değişimi iyi bir şekilde yakalayabildiği görülmektedir. Çalışmamda Random Forest modeli kullanarak gelecekteki öğretmen maaşlarını tahmin edildi. Elde edilen tablo ve grafikler, branş bazında maaş trendleri ve gelecek beklentileri hakkında değerli bilgiler sunmaktadır.

Grafiklerde 2023 yılında  beklenen maaş ile tahmini maaş tutarlı olduğu halde mevcut maaşın genel itibariyle yüksek olması, pandemi sürecinde nedeniyle oluşan yüksek enflansyondan dolayı öngörülenin dışında maaşlara 6 ayda bir zam yapılmıştır. Fakat anketler yapılmadığından verisetine yansımamıştır. Bu veri eksikliğinde dolayı 2023 yılı mevcut maaşlar, bir önceki senenin beklentilerinden ve tahminlerinden yüksek olmuştur.

Genel olarak diğer yılların beklentileri ve tahminleri tutarlıdır. Bu sonuçlar ortalama 10 yıllık tecrübeye sahip öğretmenlerin 2026 yılı için özel öğretim kursları ve kurumları ile yapacağı maaş anlaşmalarında çok iyi bir referans olacaktır.
      
        